In [1]:
!pip install tqdm ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from tqdm.notebook import tqdm
from os.path import basename, splitext
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from tempfile import mkdtemp
from ultralytics import YOLO
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Set GPU

In [3]:
# Path to CSV
result_df_path='/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/map/bihar.csv'

# Load trained YOLO model
model = YOLO("/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/yolo/models/yolo11m-obb-bihar-to-bihar/weights/best.pt")

# Name of the experiment
experiment="Thera Bihar to Test Bihar"

# GT
gt="/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/data/thera_rdn_pro/test_bihar_4x"

In [4]:
# Class information in YAML file
data_yml_save_path = mkdtemp()
data_yml = """train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_path = f"{data_yml_save_path}/data.yml"
with open(data_yml_path, "w") as f:
    f.write(data_yml)

# GT directories
gt_image_dir = gt+"/images"
gt_label_dir = gt+"/labels"
print(f"Number of images in GT: {len(os.listdir(gt_image_dir))}")

# Supervision dataset
sv_dataset = sv.DetectionDataset.from_yolo(gt_image_dir, gt_label_dir, data_yml_path)
print(f"Loaded dataset: {len(sv_dataset)} images")

# Initialize lists for predictions and targets
targets = []
predictions = []

# Iterate over the dataset and make predictions
for name,_,gt_detection in tqdm(sv_dataset):
    file_name=splitext(basename(name))[0]
    # prediction_path=join(prediction_dir,f"{file_name}.txt")
    ultralytics_result=model(name,imgsz=2560,iou=0.33,conf=0.001,exist_ok=True,save_txt=False,max_det=300, verbose=False)[0]
    sv_detection=sv.Detections.from_ultralytics(ultralytics_result)
    targets.append(gt_detection)
    predictions.append(sv_detection)

print(len(targets), len(predictions))

Number of images in GT: 687
Loaded dataset: 687 images


  0%|          | 0/687 [00:00<?, ?it/s]

687 687


In [5]:
## mAP calculation (non-class agnostic)
print (100 * "=")
print("Class-specific mAP")
print (100 * "=")
mAP_metric = MeanAveragePrecision(class_agnostic=False)
mAP_result=mAP_metric.update(predictions,targets).compute()
matched_classes=mAP_result.matched_classes.tolist()
print(f"Matched classes: {matched_classes}")

# Extract mAP values
mAP_50_95 = mAP_result.map50_95  # mAP 50:95
mAP_50 = mAP_result.map50  # mAP 50
mAP_75 = mAP_result.map75  # mAP 75
print(f"mAP 50:95: {mAP_50_95}, mAP 50: {mAP_50}, mAP 75: {mAP_75}")

# Extract class-wise mAP values
num_classes=3
final_class_wise_mAP = [0]*num_classes
class_wise_mAP=mAP_result.ap_per_class[:,0].tolist()
for cls, mAP in zip(matched_classes, class_wise_mAP):
    print(f"cls: {cls}, mAP: {mAP}")
    final_class_wise_mAP[cls] = mAP

# Compute class-agnostic mAP
print (100 * "=")
print("Class-agnostic mAP")
print (100 * "=")
mAP_metric_agnostic = MeanAveragePrecision(class_agnostic=True)
mAP_result_agnostic = mAP_metric_agnostic.update(predictions, targets).compute()
# Extract class-agnostic mAP values
mAP_50_95_agnostic = mAP_result_agnostic.map50_95  # mAP 50:95
mAP_50_agnostic = mAP_result_agnostic.map50  # mAP 50
mAP_75_agnostic = mAP_result_agnostic.map75  # mAP 75
print(f"CA mAP 50:95: {mAP_50_95_agnostic}, CA mAP 50: {mAP_50_agnostic}, CA mAP 75: {mAP_75_agnostic}")

Class-specific mAP
Matched classes: [0, 1, 2]
mAP 50:95: 0.3024563476970552, mAP 50: 0.4707268075893956, mAP 75: 0.3620919870183279
cls: 0, mAP: 0.6443132774815943
cls: 1, mAP: 0.06455290802900521
cls: 2, mAP: 0.7033142372575872
Class-agnostic mAP
CA mAP 50:95: 0.45870646398779813, CA mAP 50: 0.700746197729938, CA mAP 75: 0.5565139814530881


In [6]:
columns=["Model","CFCBK", "FCBK", "Zigzag", "Class-agnostic AP"]
result_df = pd.DataFrame(columns=columns)
new_row = [experiment] + final_class_wise_mAP + [mAP_50_agnostic]
result_df.loc[len(result_df)] = new_row  # Using loc to add the row

# Display DataFrame
display(result_df.style.hide(axis="index"))

Model,CFCBK,FCBK,Zigzag,Class-agnostic AP
Thera Bihar to Test Bihar,0.644313,0.064553,0.703314,0.700746


In [9]:
result_df.to_csv(result_df_path, index=False)

In [10]:
# Save DataFrame
result=pd.read_csv(result_df_path)
result=result.reset_index(drop=True)
result = pd.concat([result, result_df], ignore_index=True)
display(result.style.hide(axis="index"))
result.to_csv(result_df_path, index=False)

Model,CFCBK,FCBK,Zigzag,Class-agnostic AP
Thera Bihar to Test Bihar,0.644313,0.064553,0.703314,0.700746
Thera Bihar to Test Bihar,0.644313,0.064553,0.703314,0.700746


In [11]:
result=pd.read_csv(result_df_path)
display(result.style.hide(axis="index"))

Model,CFCBK,FCBK,Zigzag,Class-agnostic AP
Thera Bihar to Test Bihar,0.644313,0.064553,0.703314,0.700746
Thera Bihar to Test Bihar,0.644313,0.064553,0.703314,0.700746


In [ ]:
result=pd.read_csv(result_df_path)
result = result.iloc[:, [0, -1]]
display(result.style.hide(axis="index"))

Model,Class-agnostic AP
Thera Delhi to West Bengal,0.340852
Thera West Bengal to Delhi NCR,0.437286
